# Bias Feature Verification Analysis

This notebook visualizes the results of bias feature verification tests:
- **Suppression Test**: Setting identified bias features to 0 should reduce bias gap
- **Amplification Test**: Multiplying bias features by 2 should increase bias gap  
- **Random Control**: Suppressing random features should have minimal effect

## Validation Criteria:
1. Suppression reduces bias gap (negative change ratio)
2. Amplification increases bias gap (positive change ratio)
3. Effect is statistically significant vs random (|z-score| > 2)

## Data Source:
Results from `scripts/06_verify_bias_features.py` stored in:
- `results/{stage}/{demographic}/verification/suppression_test.json`
- `results/{stage}/{demographic}/verification/amplification_test.json`
- `results/{stage}/{demographic}/verification/random_control.json`

In [ ]:
import os
import sys
import json
import warnings
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add project root to path
NOTEBOOK_DIR = Path(os.getcwd())
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.visualization import ensure_korean_font, load_verification_results
from src.utils import load_json

warnings.filterwarnings('ignore')

print(f"Project root: {PROJECT_ROOT}")

In [ ]:
# Setup Korean font
font_name = ensure_korean_font()

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Configuration
RESULTS_DIR = PROJECT_ROOT / "results"
DATA_DIR = PROJECT_ROOT / "data"
ASSETS_DIR = PROJECT_ROOT / "notebooks" / "visualizations" / "assets"
ASSETS_DIR.mkdir(exist_ok=True, parents=True)

# Stage
STAGE = "full"

# Load demographics
demo_dict = load_json(DATA_DIR / "demographic_dict_ko.json")
DEMOGRAPHICS = list(demo_dict.keys())
DEMOGRAPHIC_EN = {d: demo_dict[d]['dimension_en'] for d in DEMOGRAPHICS}

print(f"Stage: {STAGE}")
print(f"\nDemographics ({len(DEMOGRAPHICS)}):")
for d in DEMOGRAPHICS:
    print(f"  - {d} ({DEMOGRAPHIC_EN[d]})")

---
## Data Loading Functions

In [ ]:
def load_verification_data(results_dir, stage, demographic):
    """
    Load all verification results for a demographic.
    
    Returns:
        dict with suppression, amplification, and random control data
    """
    verif_dir = results_dir / stage / demographic / 'verification'
    
    if not verif_dir.exists():
        return None
    
    result = {}
    
    # Load suppression test
    suppress_path = verif_dir / 'suppression_test.json'
    if suppress_path.exists():
        result['suppression'] = load_json(suppress_path)
    
    # Load amplification test
    amplify_path = verif_dir / 'amplification_test.json'
    if amplify_path.exists():
        result['amplification'] = load_json(amplify_path)
    
    # Load random control
    random_path = verif_dir / 'random_control.json'
    if random_path.exists():
        result['random'] = load_json(random_path)
    
    return result if result else None


print("Data loading functions defined.")

In [ ]:
# Load all verification results
verification_data = {}

print("Loading verification results...\n")

for demo in DEMOGRAPHICS:
    data = load_verification_data(RESULTS_DIR, STAGE, demo)
    if data:
        verification_data[demo] = data
        print(f"  {demo} ({DEMOGRAPHIC_EN[demo]}): Loaded")
    else:
        print(f"  {demo} ({DEMOGRAPHIC_EN[demo]}): NOT FOUND")

print(f"\nTotal demographics with verification results: {len(verification_data)}")

In [ ]:
# Create summary dataframe
summary_data = []

for demo in DEMOGRAPHICS:
    if demo not in verification_data:
        continue
    
    data = verification_data[demo]
    
    row = {
        'Demographic': demo,
        'Demographic_EN': DEMOGRAPHIC_EN[demo],
    }
    
    # Suppression data
    if 'suppression' in data:
        supp = data['suppression']
        row['Suppress_Gap_Before'] = supp.get('gap_before', 0)
        row['Suppress_Gap_After'] = supp.get('gap_after', 0)
        row['Suppress_Change_Ratio'] = supp.get('gap_change_ratio', 0)
        row['Suppress_Num_Features'] = supp.get('metadata', {}).get('num_features_manipulated', 0)
    
    # Amplification data
    if 'amplification' in data:
        amp = data['amplification']
        row['Amplify_Gap_Before'] = amp.get('gap_before', 0)
        row['Amplify_Gap_After'] = amp.get('gap_after', 0)
        row['Amplify_Change_Ratio'] = amp.get('gap_change_ratio', 0)
    
    # Random control data
    if 'random' in data:
        rand = data['random']
        row['Random_Mean_Change'] = rand.get('mean_gap_change', 0)
        row['Random_Std_Change'] = rand.get('std_gap_change', 0)
        row['Random_Num_Trials'] = rand.get('num_trials', 0)
        
        # Calculate Z-score for suppression effect
        if row.get('Suppress_Change_Ratio') is not None and rand.get('std_gap_change', 0) > 0:
            z_score = (row['Suppress_Change_Ratio'] - rand['mean_gap_change']) / rand['std_gap_change']
            row['Z_Score'] = z_score
        else:
            row['Z_Score'] = 0
    
    summary_data.append(row)

df_summary = pd.DataFrame(summary_data)

print("Verification Summary:")
print("=" * 100)
display_cols = ['Demographic_EN', 'Suppress_Change_Ratio', 'Amplify_Change_Ratio', 
                'Random_Mean_Change', 'Z_Score', 'Suppress_Num_Features']
print(df_summary[display_cols].to_string(index=False))

---
## 1. Suppression vs Amplification Comparison

Compare the gap change ratios for suppression and amplification across demographics.

In [ ]:
if len(df_summary) > 0:
    # Prepare data for plotting
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Get English labels in order
    demographics_en = df_summary['Demographic_EN'].tolist()
    
    # === Left: Gap Change Ratio Comparison ===
    ax = axes[0]
    
    x = np.arange(len(demographics_en))
    width = 0.35
    
    # Suppression bars (should be negative for success)
    suppress_ratios = df_summary['Suppress_Change_Ratio'].values * 100  # Convert to percentage
    amplify_ratios = df_summary['Amplify_Change_Ratio'].values * 100  # Convert to percentage
    
    bars1 = ax.bar(x - width/2, suppress_ratios, width, label='Suppression', 
                   color='#3498db', alpha=0.8)
    bars2 = ax.bar(x + width/2, amplify_ratios, width, label='Amplification', 
                   color='#e74c3c', alpha=0.8)
    
    # Reference line at 0
    ax.axhline(y=0, color='gray', linestyle='-', linewidth=1)
    
    ax.set_xlabel('Demographic Dimension', fontsize=12)
    ax.set_ylabel('Gap Change Ratio (%)', fontsize=12)
    ax.set_title('Bias Gap Change Ratio by Manipulation Type', fontsize=14, pad=15)
    ax.set_xticks(x)
    ax.set_xticklabels(demographics_en, rotation=45, ha='right', fontsize=10)
    ax.legend(loc='upper right', fontsize=11)
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, val in zip(bars1, suppress_ratios):
        if val != 0:
            ax.annotate(f'{val:.1f}%', xy=(bar.get_x() + bar.get_width()/2, val),
                       ha='center', va='bottom' if val > 0 else 'top', fontsize=8)
    for bar, val in zip(bars2, amplify_ratios):
        if val != 0:
            ax.annotate(f'{val:.1f}%', xy=(bar.get_x() + bar.get_width()/2, val),
                       ha='center', va='bottom' if val > 0 else 'top', fontsize=8)
    
    # === Right: Absolute Gap Values (Before/After) ===
    ax2 = axes[1]
    
    gap_before = df_summary['Suppress_Gap_Before'].values
    gap_after_suppress = df_summary['Suppress_Gap_After'].values
    gap_after_amplify = df_summary['Amplify_Gap_After'].values
    
    width = 0.25
    
    ax2.bar(x - width, gap_before, width, label='Baseline', color='gray', alpha=0.7)
    ax2.bar(x, gap_after_suppress, width, label='After Suppression', color='#3498db', alpha=0.8)
    ax2.bar(x + width, gap_after_amplify, width, label='After Amplification', color='#e74c3c', alpha=0.8)
    
    ax2.set_xlabel('Demographic Dimension', fontsize=12)
    ax2.set_ylabel('Logit Gap (Mean)', fontsize=12)
    ax2.set_title('Absolute Logit Gap Values', fontsize=14, pad=15)
    ax2.set_xticks(x)
    ax2.set_xticklabels(demographics_en, rotation=45, ha='right', fontsize=10)
    ax2.legend(loc='upper right', fontsize=10)
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(ASSETS_DIR / f"verification_comparison_{STAGE}.png", dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("No verification data available.")

---
## 2. Verification Effects by Demographic (Bar Charts)

Detailed view of baseline, suppression, amplification, and random control for each demographic.

In [ ]:
if len(verification_data) > 0:
    n_demographics = len(verification_data)
    n_cols = 3
    n_rows = (n_demographics + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
    axes = axes.flatten()
    
    fig.suptitle(
        'Bias Feature Manipulation Effects by Demographic\n'
        '(Baseline / Suppression / Amplification / Random)',
        fontsize=18,
        y=0.995
    )
    
    for i, demo in enumerate(verification_data.keys()):
        ax = axes[i]
        data = verification_data[demo]
        demo_en = DEMOGRAPHIC_EN.get(demo, demo)
        
        # Extract values
        baseline_gap = data.get('suppression', {}).get('gap_before', 0)
        suppress_gap = data.get('suppression', {}).get('gap_after', 0)
        amplify_gap = data.get('amplification', {}).get('gap_after', 0)
        
        # Random control - calculate gap from change ratio
        random_mean_change = data.get('random', {}).get('mean_gap_change', 0)
        random_gap = baseline_gap * (1 + random_mean_change)
        
        # Gap standard deviations
        baseline_std = data.get('suppression', {}).get('metadata', {}).get('gap_std_before', 0)
        suppress_std = data.get('suppression', {}).get('metadata', {}).get('gap_std_after', 0)
        amplify_std = data.get('amplification', {}).get('metadata', {}).get('gap_std_after', 0)
        random_std = data.get('random', {}).get('std_gap_change', 0) * baseline_gap
        
        # Bar data
        x_labels = ['Baseline', 'Suppress', 'Amplify', 'Random']
        means = [baseline_gap, suppress_gap, amplify_gap, random_gap]
        stds = [baseline_std, suppress_std, amplify_std, random_std]
        colors = ['gray', '#3498db', '#e74c3c', '#f39c12']
        
        x_pos = np.arange(len(x_labels))
        bars = ax.bar(x_pos, means, yerr=stds, capsize=5, color=colors, alpha=0.8)
        
        # Baseline reference line
        ax.axhline(y=baseline_gap, linestyle='--', color='gray', alpha=0.5, linewidth=1)
        
        # Labels
        ax.set_xticks(x_pos)
        ax.set_xticklabels(x_labels, fontsize=11)
        ax.set_ylabel('Logit Gap', fontsize=11)
        ax.set_title(f"{demo}\n({demo_en})", fontsize=13, pad=10)
        ax.grid(axis='y', alpha=0.3)
        
        # Add change ratio annotations
        suppress_change = data.get('suppression', {}).get('gap_change_ratio', 0) * 100
        amplify_change = data.get('amplification', {}).get('gap_change_ratio', 0) * 100
        
        if suppress_change != 0:
            color = 'green' if suppress_change < 0 else 'red'
            ax.annotate(f'{suppress_change:+.1f}%', xy=(1, suppress_gap),
                       ha='center', va='bottom', fontsize=9, color=color, fontweight='bold')
        if amplify_change != 0:
            color = 'red' if amplify_change > 0 else 'green'
            ax.annotate(f'{amplify_change:+.1f}%', xy=(2, amplify_gap),
                       ha='center', va='bottom', fontsize=9, color=color, fontweight='bold')
    
    # Hide extra subplots
    for i in range(len(verification_data), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.savefig(ASSETS_DIR / f"verification_effects_by_demo_{STAGE}.png", dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("No verification data available.")

---
## 3. Statistical Significance Analysis (Z-Score)

Compare suppression effect vs random control to assess statistical significance.

In [ ]:
if len(df_summary) > 0 and 'Z_Score' in df_summary.columns:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    demographics_en = df_summary['Demographic_EN'].tolist()
    z_scores = df_summary['Z_Score'].values
    
    # === Left: Z-Score Bar Chart ===
    ax = axes[0]
    
    colors = ['#27ae60' if abs(z) > 2 else '#e74c3c' for z in z_scores]
    
    x = np.arange(len(demographics_en))
    bars = ax.bar(x, z_scores, color=colors, alpha=0.8)
    
    # Significance thresholds
    ax.axhline(y=2, color='gray', linestyle='--', linewidth=1.5, alpha=0.7, label='p < 0.05 threshold')
    ax.axhline(y=-2, color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    
    ax.set_xlabel('Demographic Dimension', fontsize=12)
    ax.set_ylabel('Z-Score', fontsize=12)
    ax.set_title('Statistical Significance of Suppression Effect\n(vs Random Control)', fontsize=14, pad=15)
    ax.set_xticks(x)
    ax.set_xticklabels(demographics_en, rotation=45, ha='right', fontsize=10)
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, val in zip(bars, z_scores):
        ax.annotate(f'{val:.2f}', xy=(bar.get_x() + bar.get_width()/2, val),
                   ha='center', va='bottom' if val > 0 else 'top', fontsize=9, fontweight='bold')
    
    # === Right: Suppression vs Random Comparison ===
    ax2 = axes[1]
    
    suppress_changes = df_summary['Suppress_Change_Ratio'].values * 100
    random_changes = df_summary['Random_Mean_Change'].values * 100
    random_stds = df_summary['Random_Std_Change'].values * 100
    
    width = 0.35
    
    ax2.bar(x - width/2, suppress_changes, width, label='Suppression Effect', 
            color='#3498db', alpha=0.8)
    ax2.bar(x + width/2, random_changes, width, yerr=random_stds, capsize=3,
            label='Random Control (mean +/- std)', color='#f39c12', alpha=0.8)
    
    ax2.axhline(y=0, color='gray', linestyle='-', linewidth=1)
    
    ax2.set_xlabel('Demographic Dimension', fontsize=12)
    ax2.set_ylabel('Gap Change Ratio (%)', fontsize=12)
    ax2.set_title('Suppression Effect vs Random Control', fontsize=14, pad=15)
    ax2.set_xticks(x)
    ax2.set_xticklabels(demographics_en, rotation=45, ha='right', fontsize=10)
    ax2.legend(loc='upper right', fontsize=10)
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(ASSETS_DIR / f"verification_zscore_{STAGE}.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print significance summary
    significant = df_summary[abs(df_summary['Z_Score']) > 2]
    print(f"\nStatistically Significant Demographics (|z| > 2): {len(significant)}/{len(df_summary)}")
    for _, row in significant.iterrows():
        print(f"  - {row['Demographic_EN']}: z = {row['Z_Score']:.2f}")
else:
    print("No Z-score data available.")

---
## 4. Heatmap Visualization

Comprehensive heatmaps for verification metrics.

In [ ]:
if len(df_summary) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 8))
    
    # Prepare data with English labels as index
    df_heatmap = df_summary.set_index('Demographic_EN')
    
    # === Left: Gap Change Ratios Heatmap ===
    ax = axes[0]
    
    change_cols = ['Suppress_Change_Ratio', 'Amplify_Change_Ratio', 'Random_Mean_Change']
    if all(col in df_heatmap.columns for col in change_cols):
        change_data = df_heatmap[change_cols].copy() * 100  # Convert to percentage
        change_data.columns = ['Suppression', 'Amplification', 'Random']
        
        # Create custom colormap: blue (negative) - white (0) - red (positive)
        sns.heatmap(
            change_data,
            annot=True,
            fmt='.2f',
            cmap='RdBu_r',
            center=0,
            ax=ax,
            cbar_kws={'label': 'Gap Change (%)'},
            linewidths=0.5,
            annot_kws={'fontsize': 11}
        )
        
        ax.set_title('Gap Change Ratio (%)\n(Negative = Bias Reduced)', fontsize=14, pad=15)
        ax.set_xlabel('Manipulation Type', fontsize=12)
        ax.set_ylabel('Demographic', fontsize=12)
    
    # === Right: Absolute Gap Values ===
    ax2 = axes[1]
    
    gap_cols = ['Suppress_Gap_Before', 'Suppress_Gap_After', 'Amplify_Gap_After']
    if all(col in df_heatmap.columns for col in gap_cols):
        gap_data = df_heatmap[gap_cols].copy()
        gap_data.columns = ['Baseline', 'After Suppress', 'After Amplify']
        
        sns.heatmap(
            gap_data,
            annot=True,
            fmt='.4f',
            cmap='YlOrRd',
            ax=ax2,
            cbar_kws={'label': 'Logit Gap'},
            linewidths=0.5,
            annot_kws={'fontsize': 11}
        )
        
        ax2.set_title('Absolute Logit Gap Values', fontsize=14, pad=15)
        ax2.set_xlabel('Condition', fontsize=12)
        ax2.set_ylabel('Demographic', fontsize=12)
    
    plt.tight_layout()
    plt.savefig(ASSETS_DIR / f"verification_heatmaps_{STAGE}.png", dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("No data available for heatmap.")

---
## 5. Validation Criteria Summary

Check which demographics pass all three validation criteria:
1. Suppression reduces bias gap (gap_change_ratio < 0)
2. Amplification increases bias gap (gap_change_ratio > 0)
3. Effect is statistically significant vs random (|z-score| > 2)

In [ ]:
if len(df_summary) > 0:
    # Compute validation criteria
    df_validation = df_summary.copy()
    
    # Criterion 1: Suppression reduces bias (change ratio < 0)
    df_validation['C1_Suppress_Reduces'] = df_validation['Suppress_Change_Ratio'] < 0
    
    # Criterion 2: Amplification increases bias (change ratio > 0)
    df_validation['C2_Amplify_Increases'] = df_validation['Amplify_Change_Ratio'] > 0
    
    # Criterion 3: Statistically significant (|z| > 2)
    df_validation['C3_Significant'] = abs(df_validation['Z_Score']) > 2
    
    # Count criteria met
    df_validation['Criteria_Met'] = (
        df_validation['C1_Suppress_Reduces'].astype(int) +
        df_validation['C2_Amplify_Increases'].astype(int) +
        df_validation['C3_Significant'].astype(int)
    )
    
    # Create visual summary
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Prepare data for heatmap
    criteria_cols = ['C1_Suppress_Reduces', 'C2_Amplify_Increases', 'C3_Significant']
    criteria_labels = ['Suppression\nReduces Bias', 'Amplification\nIncreases Bias', 'Statistically\nSignificant']
    
    criteria_data = df_validation.set_index('Demographic_EN')[criteria_cols].astype(int)
    criteria_data.columns = criteria_labels
    
    # Custom colormap: red (fail) / green (pass)
    cmap = sns.color_palette(['#e74c3c', '#27ae60'], as_cmap=True)
    
    sns.heatmap(
        criteria_data,
        annot=True,
        fmt='d',
        cmap=cmap,
        vmin=0,
        vmax=1,
        ax=ax,
        cbar=False,
        linewidths=1,
        annot_kws={'fontsize': 14, 'fontweight': 'bold'}
    )
    
    # Replace 0/1 with symbols
    for text in ax.texts:
        if text.get_text() == '1':
            text.set_text('PASS')
            text.set_color('white')
        else:
            text.set_text('FAIL')
            text.set_color('white')
    
    # Add criteria met count as additional column
    criteria_met = df_validation.set_index('Demographic_EN')['Criteria_Met'].values
    
    ax.set_title('Validation Criteria Summary\n(0 = FAIL, 1 = PASS)', fontsize=16, pad=20)
    ax.set_xlabel('Validation Criterion', fontsize=13)
    ax.set_ylabel('Demographic', fontsize=13)
    
    plt.tight_layout()
    plt.savefig(ASSETS_DIR / f"verification_validation_summary_{STAGE}.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print text summary
    print("\n" + "=" * 70)
    print("VALIDATION SUMMARY")
    print("=" * 70)
    
    for _, row in df_validation.iterrows():
        demo = row['Demographic_EN']
        c1 = 'PASS' if row['C1_Suppress_Reduces'] else 'FAIL'
        c2 = 'PASS' if row['C2_Amplify_Increases'] else 'FAIL'
        c3 = 'PASS' if row['C3_Significant'] else 'FAIL'
        total = row['Criteria_Met']
        
        status = 'ALL PASS' if total == 3 else f'{total}/3'
        print(f"\n{demo}:")
        print(f"  [{'x' if row['C1_Suppress_Reduces'] else ' '}] Suppression reduces bias (change: {row['Suppress_Change_Ratio']*100:.2f}%)")
        print(f"  [{'x' if row['C2_Amplify_Increases'] else ' '}] Amplification increases bias (change: {row['Amplify_Change_Ratio']*100:.2f}%)")
        print(f"  [{'x' if row['C3_Significant'] else ' '}] Statistically significant (z-score: {row['Z_Score']:.2f})")
        print(f"  => Status: {status}")
    
    # Overall summary
    all_pass = len(df_validation[df_validation['Criteria_Met'] == 3])
    print(f"\n" + "=" * 70)
    print(f"OVERALL: {all_pass}/{len(df_validation)} demographics pass all criteria")
    print("=" * 70)
else:
    print("No data available for validation summary.")

---
## 6. Feature Indices Analysis

Analyze the bias feature indices that were manipulated.

In [ ]:
if len(verification_data) > 0:
    # Collect feature indices for each demographic
    feature_data = []
    all_feature_indices = {}
    
    for demo in verification_data.keys():
        data = verification_data[demo]
        if 'suppression' in data:
            feature_indices = data['suppression'].get('feature_indices', [])
            all_feature_indices[demo] = feature_indices
            
            feature_data.append({
                'Demographic': demo,
                'Demographic_EN': DEMOGRAPHIC_EN.get(demo, demo),
                'Num_Features': len(feature_indices),
                'Min_Index': min(feature_indices) if feature_indices else 0,
                'Max_Index': max(feature_indices) if feature_indices else 0,
                'Mean_Index': np.mean(feature_indices) if feature_indices else 0
            })
    
    df_features = pd.DataFrame(feature_data)
    
    print("Bias Feature Indices Summary:")
    print("=" * 80)
    print(df_features.to_string(index=False))
    
    # Visualize number of features per demographic
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart of feature counts
    ax = axes[0]
    x = np.arange(len(df_features))
    ax.bar(x, df_features['Num_Features'].values, color='steelblue', alpha=0.8)
    ax.set_xlabel('Demographic', fontsize=12)
    ax.set_ylabel('Number of Bias Features', fontsize=12)
    ax.set_title('Number of Identified Bias Features per Demographic', fontsize=14, pad=15)
    ax.set_xticks(x)
    ax.set_xticklabels(df_features['Demographic_EN'].values, rotation=45, ha='right', fontsize=10)
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for i, v in enumerate(df_features['Num_Features'].values):
        ax.text(i, v + 0.5, str(v), ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # Feature index distribution
    ax2 = axes[1]
    
    # Combine all feature indices
    all_indices = []
    for indices in all_feature_indices.values():
        all_indices.extend(indices)
    
    if all_indices:
        ax2.hist(all_indices, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
        ax2.set_xlabel('Feature Index', fontsize=12)
        ax2.set_ylabel('Frequency', fontsize=12)
        ax2.set_title('Distribution of Bias Feature Indices\n(Across All Demographics)', fontsize=14, pad=15)
        ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(ASSETS_DIR / f"verification_feature_analysis_{STAGE}.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    # Feature overlap analysis
    print("\n" + "=" * 80)
    print("Feature Overlap Analysis:")
    print("=" * 80)
    
    # Count feature frequency across demographics
    from collections import Counter
    feature_counts = Counter(all_indices)
    
    # Find features shared across multiple demographics
    shared_features = {f: c for f, c in feature_counts.items() if c > 1}
    print(f"\nTotal unique features: {len(feature_counts)}")
    print(f"Features shared across 2+ demographics: {len(shared_features)}")
    
    if shared_features:
        print("\nTop 10 most shared features:")
        for feat, count in sorted(shared_features.items(), key=lambda x: -x[1])[:10]:
            print(f"  Feature #{feat}: shared across {count} demographics")
else:
    print("No verification data available.")

---
## 7. Export Summary Table

In [ ]:
if len(df_summary) > 0:
    # Create final summary table
    final_summary = df_summary[[
        'Demographic', 'Demographic_EN',
        'Suppress_Gap_Before', 'Suppress_Gap_After', 'Suppress_Change_Ratio',
        'Amplify_Gap_After', 'Amplify_Change_Ratio',
        'Random_Mean_Change', 'Random_Std_Change', 'Z_Score',
        'Suppress_Num_Features'
    ]].copy()
    
    # Add validation columns
    final_summary['C1_Suppress_Reduces'] = final_summary['Suppress_Change_Ratio'] < 0
    final_summary['C2_Amplify_Increases'] = final_summary['Amplify_Change_Ratio'] > 0
    final_summary['C3_Significant'] = abs(final_summary['Z_Score']) > 2
    final_summary['All_Criteria_Pass'] = (
        final_summary['C1_Suppress_Reduces'] & 
        final_summary['C2_Amplify_Increases'] & 
        final_summary['C3_Significant']
    )
    
    # Save to CSV
    output_path = ASSETS_DIR / f"verification_summary_{STAGE}.csv"
    final_summary.to_csv(output_path, index=False)
    print(f"Summary saved to: {output_path}")
    
    # Display final table
    print("\nFinal Verification Summary:")
    print("=" * 120)
    display_summary = final_summary[[
        'Demographic_EN', 
        'Suppress_Change_Ratio', 'Amplify_Change_Ratio',
        'Z_Score', 'Suppress_Num_Features', 'All_Criteria_Pass'
    ]].copy()
    display_summary['Suppress_Change_Ratio'] = display_summary['Suppress_Change_Ratio'].apply(lambda x: f"{x*100:.2f}%")
    display_summary['Amplify_Change_Ratio'] = display_summary['Amplify_Change_Ratio'].apply(lambda x: f"{x*100:.2f}%")
    display_summary['Z_Score'] = display_summary['Z_Score'].apply(lambda x: f"{x:.2f}")
    display_summary.columns = ['Demographic', 'Suppress Change', 'Amplify Change', 'Z-Score', 'Num Features', 'All Pass']
    print(display_summary.to_string(index=False))
else:
    print("No data available for export.")

In [ ]:
print("\n" + "=" * 70)
print("Verification Visualization Complete!")
print("=" * 70)
print(f"\nAssets saved to: {ASSETS_DIR}")
print("\nGenerated files:")
for f in sorted(ASSETS_DIR.glob(f"verification*_{STAGE}*")):
    print(f"  - {f.name}")